Refer https://www.bilibili.com/video/BV1D1421r7sj?spm_id_from=333.788.videopod.sections&vd_source=b47e02f9c38ec61f19d536cee73b2322
The parameters and structures of models are from Summit Diware

In [1]:
!pip install torchvision


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

<img src="image/cifar-100.jpg" alt="resnet" width="700"/>


In [2]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR100(root='./cifar100', download=True, transform=transform)
loader = DataLoader(train_dataset, batch_size=500, shuffle=False) # batch size

mean = torch.zeros(3)
std = torch.zeros(3)

for images, _ in loader:
    for c in range(3):
        mean[c] += images[:, c, :, :].mean()
        std[c] += images[:, c, :, :].std()

mean /= len(loader)
std /= len(loader)

print(f'Mean: {mean}')
print(f'Std: {std}')

Mean: tensor([0.5071, 0.4865, 0.4409])
Std: tensor([0.2672, 0.2563, 0.2760])


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4865, 0.4409],#Converts images from PIL or NumPy arrays to PyTorch tensors and scales pixel values from [0,255] to [0,1]
        std=[0.2672, 0.2563, 0.2760]  # Standardizes data using the provided mean and standard deviation
    )
])

train_dataset = datasets.CIFAR100(root='./cifar100', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./cifar100', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]\tLoss: {loss.item():.4f}')

In [5]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')


<!-- ![Image from URL](image/alexnet.jpg) -->
<img src="image/resnet.png" alt="vgg" width="800"/>


In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += identity 
        return self.relu(out)

In [7]:
class ModifiedResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), # 32×32
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 32×32
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 16×16

            ResidualBlock(128), # 16×16

            nn.Conv2d(128, 256, kernel_size=3, padding=1), # 16×16
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 8×8

            nn.Conv2d(256, 512, kernel_size=3, padding=1), # 8×8
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 4×4

            ResidualBlock(512), # 4×4
            nn.MaxPool2d(2, 2), # 2×2

            nn.Flatten(),
            
            # overfitting
            nn.Dropout(0.5),
            
            nn.Linear(512*2*2, 100) 
        )

    def forward(self, x):
        return self.model(x)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

model = ModifiedResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

epochs = 25

for epoch in range(1, epochs + 1):

    train(model, device, train_loader, optimizer, epoch)

    test(model, device, test_loader)
    
    scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch} completed. Current Learning Rate: {current_lr}")


Using device: cuda
Train Epoch: 1 [0/50000]	Loss: 4.6056
Train Epoch: 1 [12800/50000]	Loss: 4.0797
Train Epoch: 1 [25600/50000]	Loss: 3.8781
Train Epoch: 1 [38400/50000]	Loss: 3.6494
Test set: Average loss: 3.2029, Accuracy: 2286/10000 (22.86%)
Epoch 1 completed. Current Learning Rate: 0.001
Train Epoch: 2 [0/50000]	Loss: 3.3071
Train Epoch: 2 [12800/50000]	Loss: 2.7966
Train Epoch: 2 [25600/50000]	Loss: 2.8015
Train Epoch: 2 [38400/50000]	Loss: 2.9879
Test set: Average loss: 2.6366, Accuracy: 3357/10000 (33.57%)
Epoch 2 completed. Current Learning Rate: 0.001
Train Epoch: 3 [0/50000]	Loss: 2.8005
Train Epoch: 3 [12800/50000]	Loss: 2.7232
Train Epoch: 3 [25600/50000]	Loss: 2.4751
Train Epoch: 3 [38400/50000]	Loss: 2.1385
Test set: Average loss: 2.2836, Accuracy: 4017/10000 (40.17%)
Epoch 3 completed. Current Learning Rate: 0.001
Train Epoch: 4 [0/50000]	Loss: 2.1652
Train Epoch: 4 [12800/50000]	Loss: 2.0468
Train Epoch: 4 [25600/50000]	Loss: 2.2445
Train Epoch: 4 [38400/50000]	Loss: 2.